In [592]:
import numpy as np
import pandas as pd
import seaborn as sns

In [593]:
demo = pd.read_csv('/Users/mwilson/Documents/GitHub/surveys/data/temp/demo_data_orig.csv')
demo = demo.fillna(0)

In [594]:
demo['edu'] = demo['deg']/(demo['elem']+demo['ass']+demo['deg'])
demo['race'] = demo['white']/(demo['white']+demo['black']+demo['nat']+demo['aapi']+demo['mixed'])
demo['malefr'] = demo['male']/(demo['male']+demo['female'])
demo['femalefr'] = demo['female']/(demo['male']+demo['female'])

In [595]:
vote = pd.read_csv('/Users/mwilson/Documents/GitHub/election_forecast/data/temp/vote_data_clean.csv')

In [596]:
# fix the year format so we can work out which ones to average over

demo['years'] = demo['year'].str.split(pat='-')
def getyear1(row):
    return row['years'][0]
def getyear2(row):
    if(len(row['years'])>1):
        return row['years'][1]
    else:
        return row['years'][0]
demo['year1'] = demo.apply(getyear1, axis=1).astype(int)
demo['year2'] = demo.apply(getyear2, axis=1).astype(int)
demo['pres_year'] = 4*demo['year2'].floordiv(4)
cols_to_drop = ['year','years','year1','year2']
demo = demo.drop(cols_to_drop, axis=1)

# now drop a lot more, maybe too many

cols_to_drop = ['white','black','nat','aapi','mixed','elem','ass','deg','male','female']
demo = demo.drop(cols_to_drop, axis=1)

# remove Puerto Rico

demo = demo[demo['state']<57]

In [597]:
# average the data covering a given presidential year - can probably do better
# no data for 2004 so remove everything

demo = demo[demo['pres_year']!=2004]
vote = vote[vote['year']!=2004]
#pres_years = [2000,2004,2008,2012,2016,2020]

demo = demo.groupby(['pres_year','state','county']).mean().reset_index()

# get ready for merging with vote

demo['year'] = demo['pres_year']
demo['fips'] = 1000*demo['state']+demo['county']
demo = demo.drop(['pres_year','state','county'],axis=1)

vote['fips'] = vote['fips'].astype(int)
vote['demfr'] = vote['DEMOCRAT']/vote['totalvotes']
vote['repfr'] = vote['REPUBLICAN']/vote['totalvotes']
vote['othfr'] = vote['OTHER']/vote['totalvotes']
vote['lagdemfr'] = vote['lagDEM']/(vote['lagDEM']+vote['lagREP']+vote['lagOTH'])
vote['lagrepfr'] = vote['lagREP']/(vote['lagDEM']+vote['lagREP']+vote['lagOTH'])
vote['lagothfr'] = vote['lagOTH']/(vote['lagDEM']+vote['lagREP']+vote['lagOTH'])
vote = vote.drop(['DEMOCRAT','REPUBLICAN','OTHER','totalvotes','lagDEM','lagREP','lagOTH'],axis=1)

In [632]:
df = pd.merge(vote,demo, on=['year','fips'])
df = df.fillna(0)

# now add polling data
### add new predictors e.g. swing - not used here

def sw(v0,vs0,d,str): 
    if(str=="ups"):
        di <- d    
    elif(str=="prop"):
        di <- d*v0/vs0
    elif(str=="pw"):
        if d>0:
            di = d*(1-v0)/(1-vs0)
        else:
            di <- d*v0/vs0
    return (di) #  Estimate of district level swing

poll = pd.DataFrame({
    'year': [2000,2008, 2012, 2016, 2020],
    'pollDfracdiff': [0,0.052, -0.033, -0.020, 0.044],
    'pollRfracdiff': [0,-0.044, 0.036, -0.045, 0.004]
})


# seems not to work well, so will ignore for now or at least until we can get more data
df = pd.merge(df,poll,on='year')
df['pollupspredD'] = df['lagdemfr']+df['pollDfracdiff'] # temp fix to avoid computing overall swing
df['pollupspredR'] = df['lagrepfr']+df['pollRfracdiff'] # temp fix to avoid computing overall swing

In [633]:
rows_with_nan = df[df.isna().any(axis=1)]
print(rows_with_nan)

Empty DataFrame
Columns: [Unnamed: 0, year, fips, demfr, repfr, othfr, lagdemfr, lagrepfr, lagothfr, inc, mage, fage, edu, race, malefr, femalefr, pollDfracdiff, pollRfracdiff, pollupspredD, pollupspredR]
Index: []


In [637]:
# add interaction term

#df = df.dropna()
df['male'] = df['malefr']*df['mage']
df['female'] = df['femalefr']*df['fage']

df_train = df[df['year']<2020]
df_test = df[df['year']==2020]
cols_to_drop1 = ['demfr','repfr','othfr','year','fips','lagothfr']
cols_to_drop2 = ['inc','mage','fage','malefr','femalefr']
#cols_to_drop3 = ['edu','race','male','female']
X_train = df_train.drop(cols_to_drop1,axis=1)
X_train = X_train.drop(cols_to_drop2,axis=1)
X_train = X_train.drop(cols_to_drop3,axis=1)
X_test = df_test.drop(cols_to_drop1,axis=1)
X_test = X_test.drop(cols_to_drop2,axis=1)
X_test = X_test.drop(cols_to_drop3,axis=1)
y_train = df_train['demfr']
y_test = df_test['demfr']


#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)


In [638]:
print(df.shape)
print(df_test.shape)
print(df_train.shape)

(15572, 22)
(3115, 22)
(12457, 22)


In [646]:
# now some basic machine learning

from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
from sklearn.metrics import mean_absolute_error,mean_squared_error

mae = mean_absolute_error(y_true=y_test,y_pred=y_pred)
mse = mean_squared_error(y_true=y_test,y_pred=y_pred) #default=True

print("MAE:",mae)
print("MSE:",mse)

MAE: 0.018673323123613753
MSE: 0.000681962620979154


In [613]:
y_err = y_pred - y_test
max(abs(y_err)), min(abs(y_err))

(0.20905627041318114, 0.0014742737408598994)

In [641]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)  
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_true=y_test,y_pred=y_pred)
mse = mean_squared_error(y_true=y_test,y_pred=y_pred) #default=True

print("MAE:",mae)
print("MSE:",mse)

MAE: 0.041494311233714457
MSE: 0.003046660894738371


In [618]:
y_err = y_pred - y_test
max(abs(y_err)), min(abs(y_err))

(0.22429981240877972, 0.0003813271685854658)

In [642]:
from xgboost.sklearn import XGBRegressor
model = XGBRegressor(n_estimators=100, random_state=42) 
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_true=y_test,y_pred=y_pred)
mse = mean_squared_error(y_true=y_test,y_pred=y_pred) #default=True

print("MAE:",mae)
print("MSE:",mse)

MAE: 0.05332312348193685
MSE: 0.004633693266638119


In [620]:
y_err = y_pred - y_test
max(abs(y_err)), min(abs(y_err))

(0.23592253618303005, 9.684577951094009e-05)

In [643]:
from lightgbm import LGBMRegressor
model = LGBMRegressor(n_estimators=100, random_state=42)  
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_true=y_test,y_pred=y_pred)
mse = mean_squared_error(y_true=y_test,y_pred=y_pred) #default=True

print("MAE:",mae)
print("MSE:",mse)

[LightGBM] [Fatal] Do not support special JSON characters in feature name.


LightGBMError: Do not support special JSON characters in feature name.

In [622]:
y_err = y_pred - y_test
max(abs(y_err)), min(abs(y_err))

(0.23592253618303005, 9.684577951094009e-05)

In [644]:
from sklearn import linear_model
model = linear_model.Ridge(alpha=0.7)  
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_true=y_test,y_pred=y_pred)
mse = mean_squared_error(y_true=y_test,y_pred=y_pred) #default=True

print("MAE:",mae)
print("MSE:",mse)

MAE: 0.035171254143620274
MSE: 0.0016986463176102296


In [624]:
y_err = y_pred - y_test
max(abs(y_err)), min(abs(y_err))

(0.20922616596423188, 0.0009982371027911774)

In [645]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=5)  
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_true=y_test,y_pred=y_pred)
mse = mean_squared_error(y_true=y_test,y_pred=y_pred) #default=True

print("MAE:",mae)
print("MSE:",mse)

MAE: 0.14014924501009496
MSE: 0.03671151792519278


In [626]:
y_err = y_pred - y_test
max(abs(y_err)), min(abs(y_err))

(0.6946871771424405, 0.00015290691013999935)